In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Load the dataset
dataset = pd.read_csv('Churn_Modeling_cleaned.csv')

# Select significant features based on EDA correlation analysis
significant_features = ['NumOfProducts_2', 'Age', 'NumOfProducts_3', 'Geography_Germany', 
                       'IsActiveMember', 'NumOfProducts_4', 'ZeroBalance', 'Balance', 'Gender']
X = dataset[significant_features]
y = dataset['Exited'].values

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Set hyperparameters (adjustable based on your best results)
units_per_layer = 12  # Your best: 12 units
num_hidden_layers = 2  # Fixed to 2 layers (no option for 3)
dropout_rate = 0.25  # Your best: 0.15 dropout
learning_rate = 0.007  # Your best: 0.1 learning rate
batch_size = 32  # Your best: 32 batch size
patience = 30  # Your best: 10 patience
max_epochs = 100  # Your best: 100 epochs
validation_split = 0.2  # Your best: assumed v_split=2 was a typo for 0.2

# Build the ANN with 2 hidden layers maximum
classifier = Sequential()
classifier.add(Dense(units=units_per_layer, activation='relu', input_dim=X_train.shape[1]))  # First hidden layer
classifier.add(Dropout(dropout_rate))  # Dropout after first hidden layer
classifier.add(Dense(units=units_per_layer, activation='relu'))  # Second hidden layer
classifier.add(Dropout(dropout_rate))  # Dropout after second hidden layer
classifier.add(Dense(units=1, activation='sigmoid'))  # Output layer

# Set custom learning rate for Adam optimizer
optimizer = Adam(learning_rate=learning_rate)

# Compile the ANN with class weights and custom optimizer
class_weight = {0: 1 / 7963, 1: 1 / 2037}  # Inverse frequency weights
total = 1 / 7963 + 1 / 2037
class_weight = {0: (1 / 7963) / total, 1: (1 / 2037) / total}  # Normalize
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Set EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

# Train the ANN with EarlyStopping and validation split
classifier.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, class_weight=class_weight, 
               validation_split=validation_split, callbacks=[early_stopping])

# Predict on the test set (probabilities)
y_pred_prob = classifier.predict(X_test)

# Precision-Recall Curve for threshold optimization
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)  # Avoid division by zero
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

# Print optimal threshold with 3 decimal places
print(f"Optimal Threshold (based on max F1-Score): {optimal_threshold:.3f}")

# Set the threshold here (edit this value to test different thresholds, e.g., 0.5 or optimal_threshold)
custom_threshold = 0.6  # Default set to optimal_threshold; change to any value between 0 and 1

# Apply the custom threshold
y_pred = (y_pred_prob > custom_threshold).astype(int)

# Calculate metrics
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
spec = tn / (tn + fp) if (tn + fp) > 0 else 0
pr_auc = np.trapezoid(recall, precision)  # Using trapezoid instead of trapz

# Display Confusion Matrix as a table (copy-paste compatible for Word)
print("\nConfusion Matrix:")
cm_table = pd.DataFrame(cm, index=['Actual Stayed (0)', 'Actual Left (1)'], columns=['Predicted Stayed (0)', 'Predicted Left (1)'])
print(cm_table.to_string())

# Print metrics with 3 decimal places
print(f"\nAccuracy: {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall: {rec:.3f}")
print(f"F1-Score: {f1:.3f}")
print(f"ROC-AUC: {roc_auc:.3f}")
print(f"Specificity: {spec:.3f}")
print(f"PR-AUC: {pr_auc:.3f}")

# Display Threshold Table for 0.1 to 0.9 (copy-paste compatible for Word)
print("\nThreshold Table [0.1 - 0.9]:")
thresholds_table = []
for thresh in np.arange(0.1, 1.0, 0.1):
    y_pred_thresh = (y_pred_prob > thresh).astype(int)
    prec_thresh = precision_score(y_test, y_pred_thresh)
    rec_thresh = recall_score(y_test, y_pred_thresh)
    f1_thresh = f1_score(y_test, y_pred_thresh)
    thresholds_table.append([thresh, prec_thresh, rec_thresh, f1_thresh])

thresholds_df = pd.DataFrame(thresholds_table, columns=['Threshold', 'Precision', 'Recall', 'F1-Score'])
thresholds_df = thresholds_df.round(3)  # Round to 3 decimal places
print(thresholds_df.to_string(index=False))

Epoch 1/100


c:\Users\rcc_0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6848 - loss: 0.1881 - val_accuracy: 0.7775 - val_loss: 0.4570
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7219 - loss: 0.1720 - val_accuracy: 0.7563 - val_loss: 0.4976
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7422 - loss: 0.1668 - val_accuracy: 0.7650 - val_loss: 0.4795
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7619 - loss: 0.1627 - val_accuracy: 0.7775 - val_loss: 0.4716
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7634 - loss: 0.1614 - val_accuracy: 0.7806 - val_loss: 0.4909
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7770 - loss: 0.1606 - val_accuracy: 0.8069 - val_loss: 0.4489
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7714 - loss: 0.1585 - val_accuracy: 0.7887 - val_loss: 0.5006
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7788 - loss: 0.1589 - val_accuracy: 0.7975